In [1]:
import pandas as pd
import numpy as np
import re

import plotly.express as px
import ipywidgets as widgets

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

In [2]:
meta = pd.read_csv("./input_folder/KS/meta_final2.tsv", sep = "\t", index_col = 0)
rename_to = {v: k for k, v in meta["old_name"].to_dict().items()}

In [3]:
path = "input_folder/KS/16S_predicted.tsv.gz"
df = pd.read_csv(path, compression="gzip", header=0, sep="\t", index_col = 0)
df.sort_values("16S_rRNA_Count", ascending = False, inplace = True)
df.head(5)

,16S_rRNA_Count,metadata_NSTI
sequence,,
5727d3487e755c7535a608fe7c7c2627,11,0.010987
55a342b38d7e5865e544472e7b1acdce,10,1.386634
a0d6d35b3f49dbf5abd28b9b6478f698,10,5.402627
a3d96918559b5ec0d9206dde8a4d287c,10,2.581496
bd6b181561a0d53f96d0f0a272dd5cab,10,1.371598


In [4]:
df.loc[df["metadata_NSTI"] > 2, :].shape

(44, 2)

In [5]:
taxo = pd.read_csv("input_folder/KS/taxonomy.tsv", sep = "\t", index_col = 0)
taxo.head()

,Taxon,Confidence
Feature ID,,
ffc36e27c82042664a16bcd4d380b286,d__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacterales; f__Enterobacteriaceae; g__Escherichia-Shigella,0.956551
8b74089f59b28e641f2c8c736707f8b2,d__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Enterobacterales; f__Enterobacteriaceae,0.983065
35451ef9f1742fae59aa48d51c7075de,d__Bacteria; p__Firmicutes; c__Bacilli; o__Bacillales; f__Bacillaceae; g__Bacillus,0.999482
1a600af5baa1803c1dbb249a706fa875,d__Bacteria; p__Chloroflexi; c__Anaerolineae; o__uncultured; f__uncultured; g__uncultured,0.976027
4394468b49611f78b96b64d72449fab5,d__Bacteria; p__Firmicutes; c__Bacilli; o__Lactobacillales; f__Lactobacillaceae; g__Lactobacillus; s__Lactobacillus_fermentum,0.897128


In [6]:
sorted(
    taxo.loc[
        df.loc[df["metadata_NSTI"] > 2, :].index.tolist(), :
    ].value_counts("Taxon").index.tolist()
)

['d__Archaea',
 'd__Archaea; p__Aenigmarchaeota; c__Aenigmarchaeia; o__Aenigmarchaeales; f__Aenigmarchaeales; g__Aenigmarchaeales; s__uncultured_archaeon',
 'd__Archaea; p__Aenigmarchaeota; c__Aenigmarchaeia; o__Aenigmarchaeales; f__Aenigmarchaeales; g__Candidatus_Aenigmarchaeum; s__uncultured_archaeon',
 'd__Archaea; p__Crenarchaeota; c__Bathyarchaeia; o__Bathyarchaeia; f__Bathyarchaeia; g__Bathyarchaeia; s__uncultured_archaeon',
 'd__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o__Woesearchaeales; f__GW2011_GWC1_47_15; g__GW2011_GWC1_47_15; s__uncultured_archaeon',
 'd__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o__Woesearchaeales; f__SCGC_AAA011-D5; g__SCGC_AAA011-D5',
 'd__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o__Woesearchaeales; f__SCGC_AAA011-D5; g__SCGC_AAA011-D5; s__Candidatus_Pacearchaeota',
 'd__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o__Woesearchaeales; f__SCGC_AAA011-D5; g__SCGC_AAA011-D5; s__uncultured_archaeon',
 'd__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o

In [7]:
a = pd.DataFrame(
    taxo.loc[
            df.loc[df["metadata_NSTI"] > 2, :].index.tolist(), :
        ].value_counts("Taxon")
)

a.columns = ["Number of ASVs"]
a = a.loc[sorted(a.index.tolist()), :]
a

,Number of ASVs
Taxon,
d__Archaea,3
d__Archaea; p__Aenigmarchaeota; c__Aenigmarchaeia; o__Aenigmarchaeales; f__Aenigmarchaeales; g__Aenigmarchaeales; s__uncultured_archaeon,4
d__Archaea; p__Aenigmarchaeota; c__Aenigmarchaeia; o__Aenigmarchaeales; f__Aenigmarchaeales; g__Candidatus_Aenigmarchaeum; s__uncultured_archaeon,2
d__Archaea; p__Crenarchaeota; c__Bathyarchaeia; o__Bathyarchaeia; f__Bathyarchaeia; g__Bathyarchaeia; s__uncultured_archaeon,1
d__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o__Woesearchaeales; f__GW2011_GWC1_47_15; g__GW2011_GWC1_47_15; s__uncultured_archaeon,1
d__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o__Woesearchaeales; f__SCGC_AAA011-D5; g__SCGC_AAA011-D5,4
d__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o__Woesearchaeales; f__SCGC_AAA011-D5; g__SCGC_AAA011-D5; s__Candidatus_Pacearchaeota,1
d__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o__Woesearchaeales; f__SCGC_AAA011-D5; g__SCGC_AAA011-D5; s__uncultured_archaeon,1
d__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o__Woesearchaeales; f__SCGC_AAA011-D5; g__SCGC_AAA011-D5; s__uncultured_euryarchaeote,4


In [10]:
pca_df = pd.read_csv("input_folder/KS/functional_metagenome/KO/pca.tsv", sep = "\t")
metadata_dropdown = widgets.Dropdown(options=list(meta.columns), description='Select Metadata:')

def update_color(selected_metadata):
    fig = px.scatter(pca_df, x='PC1', y='PC2',
                     color = selected_metadata,
                     text='uniqID', title='PCA Analysis')
    fig.update_traces(textposition='top center')
    fig.update_layout(width = 700, height = 600)
    fig.show()

# Observe the changes and update the plot on metadata selection
widgets.interactive(update_color, selected_metadata=metadata_dropdown)

interactive(children=(Dropdown(description='Select Metadata:', options=('uniqID', 'filename_prefix', 'temperat…

In [12]:
pca_df = pd.read_csv("input_folder/KS/functional_metagenome/KO/bray_pca.tsv", sep = "\t")
metadata_dropdown = widgets.Dropdown(options=list(meta.columns), description='Select Metadata:')

def update_color(selected_metadata):
    fig = px.scatter(pca_df, x='PC1', y='PC2',
                     color = selected_metadata,
                     text='uniqID', title='Bray Curtis + PCA Analysis')
    fig.update_traces(textposition='top center')
    fig.update_layout(width = 700, height = 600)
    fig.show()

# Observe the changes and update the plot on metadata selection
widgets.interactive(update_color, selected_metadata=metadata_dropdown)

interactive(children=(Dropdown(description='Select Metadata:', options=('uniqID', 'filename_prefix', 'temperat…

In [13]:
pca_df = pd.read_csv("input_folder/KS/functional_metagenome/KO/umap.tsv", sep = "\t")
metadata_dropdown = widgets.Dropdown(options=list(meta.columns), description='Select Metadata:')

def update_color(selected_metadata):
    fig = px.scatter(pca_df, x='UMAP1', y='UMAP2',
                     color = selected_metadata,
                     text='uniqID', title='UMAP Analysis')
    fig.update_traces(textposition='top center')
    fig.update_layout(width = 700, height = 600)
    fig.show()

# Observe the changes and update the plot on metadata selection
widgets.interactive(update_color, selected_metadata=metadata_dropdown)

interactive(children=(Dropdown(description='Select Metadata:', options=('uniqID', 'filename_prefix', 'temperat…

In [15]:
pca_df = pd.read_csv("input_folder/KS/functional_metagenome/KO/bray_umap.tsv", sep = "\t")
metadata_dropdown = widgets.Dropdown(options=list(meta.columns), description='Select Metadata:')

def update_color(selected_metadata):
    fig = px.scatter(pca_df, x='UMAP1', y='UMAP2',
                     color = selected_metadata,
                     text='uniqID', title='Bray Curtis + UMAP Analysis')
    fig.update_traces(textposition='top center')
    fig.update_layout(width = 700, height = 600)
    fig.show()

# Observe the changes and update the plot on metadata selection
widgets.interactive(update_color, selected_metadata=metadata_dropdown)

interactive(children=(Dropdown(description='Select Metadata:', options=('uniqID', 'filename_prefix', 'temperat…